Este archivo contiene la parte práctica de Reasignación de Frecuencias propuesto para Innovación Emergente

Ojo que todo lo que use nota.sintetizar3 en adelante (anterior a Best_Match v4) sintetiza mal, una 8va arriba

In [ ]:
#Correr esto para importar la librería y etc
import os
os.chdir(r'C:\Users\Graciela\Desktop\Cochlear Implants\CODE ultima version')
%run "CIS_6_Best_Match v3.ipynb"

MISMATCH = 0
CHANNEL_INTERACTION_INDEX = 0
DISPERTION = 0.001
WIN_LEN_ACE = 100
MODE = CONTINOUS
MAXIMA = 5
ORDEN = 6

GENERIC_MEDEL3 = Elec_Array(N=12, xi = 28, dx = 23.1/11) #El de Francisco, Flex28
GENERIC_MEDEL3.freqsAnalisis = [120,235,384,579,836, 1175,1624,2222,3019,4084,5507,7410]
GENERIC_MEDEL3.name = 'Medel Francisco Opus 2 FLEX28'
GENERIC_MEDEL3.manufacturer = MEDEL
GENERIC_MEDEL3.channel1_lowcut = 70 #FSP = 70Hz, HDCIS = 250Hz

GENERIC_MEDEL3.setearParametros()
elec_array = GENERIC_MEDEL3



GENERIC_MEDEL6 = Elec_Array(N=12, xi = 31.5, dx=26.4/11) #monopolar  Jace Wolfe p339
GENERIC_MEDEL6.freqsAnalisis = freqs_estandar
GENERIC_MEDEL6.name = 'Standard'
GENERIC_MEDEL6.manufacturer = MEDEL

GENERIC_COCHLEAR3 = Elec_Array(N=22, xi= 25, dx = 20/21 )
GENERIC_COCHLEAR3.freqsAnalisis = freqTable22
GENERIC_COCHLEAR3.name = 'Slim Straight (xi = 25mm)'
GENERIC_COCHLEAR3.manufacturer = COCHLEAR


saw = [1/i for i in range(1,10)]
tri = [1/(i**2) for i in range(1,10)]
sq =  [1/i for i in range(1,10)]
for i in range(len(sq)): 
    if i %2 ==1: sq[i] = 0.

In [ ]:
print(GENERIC_MEDEL3.freqsAnalisis)
print()
print(np.round(GENERIC_MEDEL3.freqsElectrodos))

In [ ]:
def test_suite_Nota(nota,elec_array, nChann,max_di):

    sig =  pureNote(nota.freq, nota.amps, 2, elec_array.SR )*10

    print('Señal original')
    plt.plot(sig[0:100])
    display(Audio(sig, rate= elec_array.SR))
    plot_stft_señalsola(sig, elec_array.freqsElectrodos)

    print('Señal sintetizada para que elicite esa altura')
    A = elec_array.generarSenialMejorMatchNota(nota.freq, nota.amps, nChann,max_di, 2)
    display(Audio(A, rate = elec_array.SR))
    #plot_stft_señalsola(A)
    plot_stft(A, elec_array, analisis = True, mini = True)

    print('Señal que daría el implante')
    out = elec_array.CIS(A)
    display(Audio(out, rate = OUT_SR))
    plot_stft(out,elec_array, analisis = False, mini = True)

## Prueba preliminar 1
Chequeo de sanidad : 1 armónico perfectamente alineado

In [ ]:
MISMATCH = 0
CHANNEL_INTERACTION_INDEX = 0
DISPERTION = 0.001
WIN_LEN_ACE = 1000
MODE = CONTINOUS
MAXIMA = 5
ORDEN = 6

#Parámetros de la prueba
nChann = 1
max_di = 700

fund = 893
amp_armonicos = [1]
dur_seg = 2
sig = 0.5* np.sum([puretone(fund*(i+1), dur_seg, IMPLANT_SR )*amp_armonicos[i] for i in range(len(amp_armonicos))] , axis=0)


#Nota:
nota = Nota(fund, amp_armonicos, dur_seg)

print('Señal con altura original (puede ser representada por el 4to canal) \n Amarillo: Frecuencia de los electrodos')
display(Audio(sig, rate=IMPLANT_SR))
plot_stft_señalsola(sig, GENERIC_MEDEL3.freqsElectrodos)

print('Señal sintetizada para que elicite esa altura (estimula el 4to canal de análisis)')
A = GENERIC_MEDEL3.generarSenialMejorMatchNota(nota, nChann,max_di)
display(Audio(A, rate = IMPLANT_SR))
#plot_stft_señalsola(A)
plot_stft(A, GENERIC_MEDEL3, analisis = True, mini = True)

print('Señal que daría el implante (estimula el 4to canal)')
out = GENERIC_MEDEL3.CIS(A)
display(Audio(out, rate = OUT_SR))
plot_stft(out,GENERIC_MEDEL3, analisis = False, mini = True)
print()

Vemos ya un problema: Aunque la señal tenga, en teoría, un solo "canal", lo que escucha la persona son varios canales. Esto es porque los filtros *de análisis* no son lo suficientemente abruptos.
Veamos que sucede en cambio si cambiamos el orden:

In [ ]:
ORDEN = 12

print('Señal que daría el implante')
out = GENERIC_MEDEL3.CIS(A)
display(Audio(out, rate = OUT_SR))
plot_stft(out,GENERIC_MEDEL3, analisis = False, mini = True)
print()

Y, sí, nos da algo más pronunciado

## Prueba Preliminar 1b
Una frecuencia, cercana a un electrodo
Debería volver a darme el canal 4 como activado, pero esta vez, ya no es una altura cercana a la original

In [ ]:
fund = 1050
ORDEN = 12
DISPERTION  = 0.01
MAXIMA  = 1

sig = 0.5* np.sum([puretone(fund*(i+1), dur_seg, IMPLANT_SR )*amp_armonicos[i] for i in range(len(amp_armonicos))] , axis=0)

print('Señal con altura original (entre 4to y 5to canal, más cerca del 4to)')
display(Audio(sig, rate=IMPLANT_SR))
plot_stft_señalsola(sig, GENERIC_MEDEL3.freqsElectrodos)

print('Señal sintetizada para que excite al canal más cercano')
A = GENERIC_MEDEL3.generarSenialMejorMatchNota(nota, 1,700)
display(Audio(A, rate = IMPLANT_SR))
plot_stft(A, GENERIC_MEDEL3, analisis = True, mini = True)

print('Señal que daría el implante (estimula el 4to canal)')
out = GENERIC_MEDEL3.CIS(A)
display(Audio(out, rate = OUT_SR))
plot_stft(out,GENERIC_MEDEL3, analisis = False, mini = True)
print()

## Prueba Preliminar 2
Dos armónicos, en relación de 8va

In [ ]:
ORDEN = 20

nChann = 2
max_di = 50

fund = 870
amp_armonicos = [1,1]
dur_seg = 2
sig = 0.5* np.sum([puretone(fund*(i+1), dur_seg, IMPLANT_SR )*amp_armonicos[i] for i in range(len(amp_armonicos))] , axis=0)

nota = Nota(fund, amp_armonicos, dur_seg)

print('Señal con altura original (parciales cerca del 4to y 6to canal)')
display(Audio(sig, rate=IMPLANT_SR))
plot_stft_señalsola(sig, GENERIC_MEDEL3.freqsElectrodos)

print('Señal sintetizada para que elicite esa altura (estimula el 4to y 6to canal de análisis)')
A = GENERIC_MEDEL3.generarSenialMejorMatchNota(nota, nChann,max_di)
display(Audio(A, rate = IMPLANT_SR))
#plot_stft_señalsola(A)
plot_stft(A, GENERIC_MEDEL3, analisis = True, mini = True)

print('Señal que daría el implante (estimula el 4to y 6to canal. EL error es < 50 cents')
out = GENERIC_MEDEL3.CIS(A)
display(Audio(out, rate = OUT_SR))
plot_stft(out,GENERIC_MEDEL3, analisis = False, mini = True)
print()

# Gráficos JJI 

## Gráfico 1 - Veo electrodos activados Con mi programa

In [ ]:
nota.parciales

In [ ]:
fund = 870
amp_armonicos = [1,1, 0,0,1]
dur_seg = 2
nota = Nota(fund, amp_armonicos, dur_seg, [])
sig = nota.sintetizar(elec_array.SR)

print('Señal con altura original (parciales cerca del 4to y 6to canal)')
display(Audio(sig, rate=IMPLANT_SR))
plot_stft_señalsola(sig, elec_array.freqsElectrodos)


Calculo electrodos activados normalmente

In [ ]:
MISMATCH = 0
CHANNEL_INTERACTION_INDEX = 0
DISPERTION = 0.001
WIN_LEN_ACE = 1000
MODE = CONTINOUS
MAXIMA = 5
ORDEN = 20

elec_array = GENERIC_MEDEL3
out_normal = elec_array.CIS(sig)
display(Audio(out_normal, rate=OUT_SR))

Reasigno frecuencias para activar los que yo quiero

In [ ]:
nChann = 4
max_di = 50

print('Señal sintetizada para elicitar esa altura (estimula el 4to y 6to canal de análisis)')
out_reasignada = elec_array.generarSenialMejorMatchNota(nota, nChann,max_di)
display(Audio(out_reasignada, rate = elec_array.SR))
#plot_stft_señalsola(A)
plot_stft(out_reasignada, elec_array, analisis = True, mini = True)

print('Señal que daría el implante (estimula el 4to y 6to canal. EL error es < 50 cents para los primeros 2')
out = elec_array.CIS(out_reasignada)
display(Audio(out, rate = OUT_SR))
plot_stft(out,elec_array, analisis = False, mini = True)
print()

Grafico juntas las FFT (se re superponen... descarto esta idea)

In [ ]:

from scipy.fft import rfft, rfftfreq

#Generales
f_from = 200
f_to = 10000
range_dB = 50 
norm = 1
logx = True

sr = OUT_SR
T = 1/sr #periodo
N = len(sig)
t = np.arange(0, N*T, T) # Sampling times

# FFT and bin centers
f = rfftfreq(N, T)
#df : sr/N  --> f(i) = i* sr/N --> i = f*N/sr  
if(f_to == 0): fto = sr/2
ifrom = round(f_from*N/sr)
ito = round(f_to*N/sr)

# Plot spectrum
fig, ax = plt.subplots(dpi = 200)
if(logx): ax.set_xscale('log')


#Plot señales
for sigcolor_pair in [(sig,'black') , (out_normal, 'red'), (out_reasignada, 'blue')]:
 
  sig, color = sigcolor_pair[0] , sigcolor_pair[1]

  y = rfft(sig)
  y = np.abs(y)*2/N
  y /= norm
  #y = 20*np.log10(np.abs(y)) #dB
  y_db = 20*np.log10(y)

  ax.plot(f[ifrom:ito], y_db[ifrom:ito], color = color, linewidth = 0.5) #eje lineal, y logaritmico
  y_max =  max(y) +5
  y_min = max(y) -range_dB
  ax.set_ylim(y_min , y_max)

ax.legend((f'I = {CHANNEL_INTERACTION_INDEX}, A = 0', f'I = {CHANNEL_INTERACTION_INDEX}, A = {ASYMMETRY_INDEX}'), loc="upper right")

plt.xlabel('Frecuencia (Hz)')
plt.ylabel('Amplitud (dB)')  
plt.grid(alpha  = 0.8)


Grafico con puntos. Intento juntar los tres espectrogramas de linea arriba en uno solo

In [ ]:
#####################################
#Electrodos:
#Medio:  Nota y sus armónicos
#Abajo: Electrodos activados

freqs_electrodos = elec_array.freqsElectrodos
y1 = [ 0.7 for f in freqs_electrodos]

freqs_nota = nota.parciales
y2 = [ 0.45 for f in freqs_nota]

freqs_activadas = nota.parciales
y3 = [0.2 for f in freqs_activadas]

###########

import matplotlib.ticker as ticker
xmin = 220
xmax = 13000

fig, ax = plt.subplots(figsize=(16,6), dpi= 130)
ax.set_xscale('log')
ax.set_xlabel('Frecuencia (Hz)', labelpad = 15,fontdict={'size':25} )
ax.xaxis.set_tick_params(pad = 18)
ax.set_xlim(xmin , xmax)
ax.set_ylim(0 , 1)
ax.set_yticks([])
ax.set_xticks([])
ax.hlines(y=[0.7,0.45, 0.2], xmin=xmin, xmax=xmax, color='gray', alpha=0.85, linewidth=1, linestyles='dashdot')

#cambiar esto a Title
ax.set_title('Electrodos activados para una nota de frecuencia 870Hz', color = 'k', style = 'italic', pad = 20, fontdict={'size':32, 'font': 'Times New Roman'}, )

#Texto para cada marquita de los elec
elec_labels = ['$e_{' +str(i) +'}$ ' for i in range(elec_array.Nchann) ]
act_labels  = ['$e_{' +str(i) +'}$ ' for i in [3,4,8] ]

#############################################3


step = 1

tuple2 = (freqs_nota, 'maroon', 'Componentes de la nota objetivo', _, y2)
tuple3 = (freqs_activadas, 'navy', 'Electrodos activados', act_labels, y3)

#Grafico las tres lineas de puntitos
for freq_color_label_ticklabels_y in [tuple1, tuple2 , tuple3]:

    (freqs, color, label, ticklabels, y) = freq_color_label_ticklabels_y #unpack

    ax.scatter(y=y, x=freqs, s=75, color=color, alpha=0.85, label = label) #Los puntitos
    ax.grid(alpha = 0.7)


#Eje de arriba y de abajo
for pair in  [(tuple1, 'top') , (tuple3, 'bottom')]:

    freq_color_label_ticklabels_y, pos = pair[0], pair[1]
    (freqs, color, label, ticklabels, y) = freq_color_label_ticklabels_y

    secax = ax.secondary_xaxis(pos) #eje secundario arriba    
    secax_freqs = ax.secondary_xaxis(pos) #eje secundario arriba    

    freq_labels = [f'${round(freq)}$ ' for freq in freqs ]

    secax.set_xticks(freqs[::step], labels = ticklabels[::step])
    secax.xaxis.set_tick_params(pad = 25, labelcolor = color, labelsize = 20, rotation = 30)
    secax_freqs.set_xticks(freqs, labels = freq_labels[::step])
    secax_freqs.xaxis.set_tick_params(pad = 6, labelcolor = color, labelsize= 20)
    for i in range(1,len(freqs)//step+1):
        secax.get_xticklabels()[-i].set_rotation_mode('anchor')

#ax.set_xticks([], minor=True)




######################################

# Title, Label, Ticks and Ylim
#ax.set_yticks(df.index)
#ax.set_yticklabels(df.manufacturer.str.title(), fontdict={'horizontalalignment': 'right'})
plt.legend(framealpha=1, shadow=True,  loc = 'upper right',  prop={'size': 20})
plt.show()

Lo mejoro para resultados generales (no le meto a mano el resultado sino que va parametrizado)

In [ ]:
fund = 870
amp_armonicos = [1,1, 1,1,1, 1,1,1]
dur_seg = 2
nota = Nota(fund, amp_armonicos, dur_seg, [])
elec_array = GENERIC_MEDEL3

nChann = 4
max_di = 50

mejorPatron = encontrarMejorMatch2(nota, elec_array.freqsElectrodos, nChann, max_di)
mejorPatron.print()

In [ ]:


#####################################
#Electrodos: Arriba
#Medio:  Nota y sus armónicos
#Abajo: Electrodos activados


freqs_electrodos = elec_array.freqsElectrodos
y1 = [ 0.6 for f in freqs_electrodos]

freqs_nota = nota.parciales
y2 = [ 0.45 for f in freqs_nota]

freqs_activadas = [elec.freq for elec in mejorPatron.canalesActivos()]
y3 = [0.3 for f in freqs_activadas]

###########

import matplotlib.ticker as ticker
xmin = 220
xmax = 13000

fig, ax = plt.subplots(figsize=(16,6), dpi= 130)
ax.set_xscale('log')
ax.set_xlabel('Frecuencia (Hz)', labelpad = 35,fontdict={'size':25} )
ax.xaxis.set_tick_params(pad = 7)
ax.set_xlim(xmin , xmax)
ax.set_ylim(0.1 , 1)
ax.set_yticks([])
ax.set_xticks([])
ax.hlines(y=[0.6,0.45, 0.3], xmin=xmin, xmax=xmax, color='gray', alpha=0.85, linewidth=1, linestyles='dashdot')
ax.vlines(x=freqs_electrodos, ymin=0, ymax=1, color='gray', alpha=0.6, linewidth=1.2, linestyles='dashdot')
ax.vlines(x=freqs_electrodos, ymin=y3[0], ymax=y1[0], color='darkgray', alpha=0.95, linewidth=2, linestyles='dashdot') #Igual pero más corta y marcada

#cambiar esto a Title
ax.set_title('Electrodos activados con el esquema de reasignación', color = 'k', style = 'italic', pad = 20, fontdict={'size':32, 'font': 'Times New Roman'}, )

#Texto para cada marquita de los elec
elec_labels = ['$e_{' +str(i+1) +'}$ ' for i in range(elec_array.Nchann) ]
act_labels  = ['$e_{' +str(i+1) +'}$ ' for i in [3,4,8] ]

#############################################3
step = 1

tuple1 = (freqs_electrodos, 'navy', 'Frecuencias de electrodos', elec_labels, y1)
tuple2 = (freqs_nota, 'maroon', 'Componentes de la nota objetivo', _, y2)
tuple3 = (freqs_activadas, 'darkgreen', 'Electrodos activados', act_labels, y3)

#Grafico las tres lineas de puntitos
for freq_color_label_ticklabels_y in [tuple1, tuple2 , tuple3]:

    (freqs, color, label, ticklabels, y) = freq_color_label_ticklabels_y #unpack

    ax.scatter(y=y, x=freqs, s=300, color=color, alpha=0.85, label = label) #Los puntitos
    ax.grid(alpha = 0.7)

#Para los electodos apagados en la linea 3, repito más transparente la última linea y le agrego cositas:
ax.scatter(y=[y3[0] for f in freqs_electrodos], x=freqs_electrodos, s=125, color='navy', alpha=0.45) #Los puntitos chiquitos y transparentes
ax.scatter(y=y3, x=freqs_activadas, s=125, color='yellow', alpha=0.9, marker = '2') 

#Eje de arriba y de abajo
for pair in  [(tuple1, 'top') , (tuple3, 'bottom')]:

    freq_color_label_ticklabels_y, pos = pair[0], pair[1]
    (freqs, color, label, ticklabels, y) = freq_color_label_ticklabels_y

    secax = ax.secondary_xaxis(pos) #eje secundario arriba    
    secax_freqs = ax.secondary_xaxis(pos) #eje secundario arriba    

    freq_labels = [f'${round(freq)}$ ' for freq in freqs ]

    secax.set_xticks(freqs[::step], labels = ticklabels[::step])
    secax.xaxis.set_tick_params(pad = 25, labelcolor = color, labelsize = 20, rotation = 30)
    secax_freqs.set_xticks(freqs, labels = freq_labels[::step])
    secax_freqs.xaxis.set_tick_params(pad = 7, labelcolor = color, labelsize= 20)
    for i in range(1,len(freqs)//step+1):
        secax.get_xticklabels()[-i].set_rotation_mode('anchor')

#ax.set_xticks([], minor=True)


######################################

# Title, Label, Ticks and Ylim
#ax.set_yticks(df.index)
#ax.set_yticklabels(df.manufacturer.str.title(), fontdict={'horizontalalignment': 'right'})
plt.legend(framealpha=1, shadow=True,  loc = 'upper right',  prop={'size': 20})
plt.show()

Repito sin la primer fila, me di cuenta después que va a ser más claro de esta forma

In [ ]:
fund = 870
amp_armonicos = [1,1, 1,1,1, 1,1,1]
dur_seg = 2
nota = Nota(fund, amp_armonicos, dur_seg, [])
elec_array = GENERIC_MEDEL3

nChann = 4
max_di = 50

mejorPatron = encontrarMejorMatch2(nota, elec_array.freqsElectrodos, nChann, max_di)
mejorPatron.print()

elecs_activados = []
for i, elec in enumerate(mejorPatron.elecs):
    if elec.on: elecs_activados.append(i)


In [ ]:


#####################################
#Electrodos: Arriba
#Medio:  Nota y sus armónicos
#Abajo: Electrodos activados


freqs_electrodos = elec_array.freqsElectrodos
y1 = [ 0.6 for f in freqs_electrodos]

freqs_nota = nota.parciales
y2 = [ 0.45 for f in freqs_nota]

freqs_activadas = [elec.freq for elec in mejorPatron.canalesActivos()]
y3 = [0.3 for f in freqs_activadas]

###########

import matplotlib.ticker as ticker
xmin = 700
xmax = 9000

fig, ax = plt.subplots(figsize=(16,6), dpi= 130)
ax.set_xscale('log')
ax.set_xlabel('Frecuencia (Hz)', labelpad = 35,fontdict={'size':25} )
ax.xaxis.set_tick_params(pad = 7)
ax.set_xlim(xmin , xmax)
ax.set_ylim(0.1 , 0.8)
ax.set_yticks([])
ax.set_xticks([])
ax.hlines(y=[0.45, 0.3], xmin=xmin, xmax=xmax, color='gray', alpha=0.85, linewidth=1, linestyles='dashdot')
ax.vlines(x=freqs_electrodos, ymin=0, ymax=1, color='gray', alpha=0.6, linewidth=1.2, linestyles='dashdot')
ax.vlines(x=freqs_electrodos, ymin=y3[0], ymax=y1[0], color='darkgray', alpha=0.95, linewidth=2, linestyles='dashdot') #Igual pero más corta y marcada

#cambiar esto a Title
ax.set_title('Electrodos seleccionados utilizando el esquema de reasignación', color = 'k', style = 'italic', pad = 20, fontdict={'size':32, 'font': 'Times New Roman'}, )

#Texto para cada marquita de los elec
elec_labels = ['$e_{' +str(i+1) +'}$ ' for i in range(elec_array.Nchann) ]
act_labels  = ['$e_{' +str(i+1) +'}$ ' for i in elecs_activados ]

#############################################3
step = 1

tuple1 = (freqs_electrodos, 'navy', 'Frecuencias de electrodos', elec_labels, y1, 'x')
tuple2 = (freqs_nota, 'maroon', 'Armónicos de la nota objetivo', _, y2, 's')
tuple3 = (freqs_activadas, 'darkgreen', 'Electrodos seleccionados', act_labels, y3, 'o')

#Grafico las tres lineas de puntitos
for freq_color_label_ticklabels_y_forma in [tuple2 , tuple3]:

    (freqs, color, label, ticklabels, y, formita) = freq_color_label_ticklabels_y_forma #unpack

    ax.scatter(y=y, x=freqs, s=600, color=color, alpha=0.9, label = label ,marker = formita) #Los puntitos
    ax.grid(alpha = 0.7)

#Para los electodos apagados en la linea 3, repito más transparente la última linea y le agrego cositas:
ax.scatter(y=[y3[0] for f in freqs_electrodos], x=freqs_electrodos, s=125, color='navy', alpha=0.45) #Los puntitos chiquitos y transparentes
ax.scatter(y=y3, x=freqs_activadas, s=125, color='yellow', alpha=0.9, marker = '2') 

#Eje de arriba: Toda la info
(freqs, color, label, ticklabels, y, formita) = tuple1

secax = ax.secondary_xaxis('top')  
secax_freqs = ax.secondary_xaxis('top')  
freq_labels = [f'${round(freq)}$ ' for freq in freqs ]
secax.set_xticks(freqs[::step], labels = ticklabels[::step])
secax.xaxis.set_tick_params(pad = 33, labelcolor = color, labelsize = 20, rotation = 30)
secax_freqs.set_xticks(freqs, labels = freq_labels[::step])
secax_freqs.xaxis.set_tick_params(pad = 7, labelcolor = color, labelsize= 25)
for i in range(1,len(freqs)//step+1):
    secax.get_xticklabels()[-i].set_rotation_mode('anchor')

#Eje de abajo: Solo electrodos activados
(freqs, color, label, ticklabels, y, formita) = tuple3
secax = ax.secondary_xaxis('bottom')  
secax.set_xticks(freqs[::step], labels = ticklabels[::step])
secax.xaxis.set_tick_params(pad = 5, labelcolor = color, labelsize = 30, rotation = 30)
for i in range(1,len(freqs)//step+1):
    secax.get_xticklabels()[-i].set_rotation_mode('anchor')


#Freqs de los armónicos
for i, freq in enumerate(nota.parciales):
    plt.text(freq, 0.5-0.13*(i%2), int(freq), fontdict={'size':30, 'font': 'Times New Roman', 'color':'maroon'})
    plt.text(freq*0.98, 0.435, int(i+1), fontdict={'size':25, 'font': 'Times New Roman', 'color':'yellow'})


#ax.set_xticks([], minor=True)


######################################

# Title, Label, Ticks and Ylim
#ax.set_yticks(df.index)
#ax.set_yticklabels(df.manufacturer.str.title(), fontdict={'horizontalalignment': 'right'})
plt.legend(framealpha=1, shadow=True,  loc = 'upper right',  prop={'size': 24})
plt.show()

## Gráfico 1 hecho función

Excelente. Ahora hago esto una función para reutilizarlo y hacer gráficos más fácil. 

In [ ]:
def grafico1(elec_array, nota, nChann, max_di, step, xmin= 500, xmax = 9000):

    mejorPatron = encontrarMejorMatch2(nota, elec_array.freqsElectrodos, nChann, max_di)
    mejorPatron.print()

    elecs_activados = []
    for i, elec in enumerate(mejorPatron.elecs):
        if elec.on: elecs_activados.append(i)

    if elecs_activados == []: return
    
    #####################################
    #Medio:  Nota y sus armónicos
    #Abajo: Electrodos activados

    freqs_electrodos = elec_array.freqsElectrodos
    y1 = [ 0.6 for f in freqs_electrodos]

    freqs_nota = nota.parciales
    y2 = [ 0.45 for f in freqs_nota]

    freqs_activadas = [elec.freq for elec in mejorPatron.canalesActivos()]
    y3 = [0.3 for f in freqs_activadas]

    ###########

    import matplotlib.ticker as ticker

    fig, ax = plt.subplots(figsize=(16,6), dpi= 130)
    ax.set_xscale('log')
    ax.set_xlabel('Frecuencia (Hz)', labelpad = 35,fontdict={'size':25} )
    ax.xaxis.set_tick_params(pad = 7)
    ax.set_xlim(xmin , xmax)
    ax.set_ylim(0.1 , 0.8)
    ax.set_yticks([])
    ax.set_xticks([])
    ax.hlines(y=[0.45, 0.3], xmin=xmin, xmax=xmax, color='gray', alpha=0.85, linewidth=1, linestyles='dashdot')
    ax.vlines(x=freqs_electrodos, ymin=0, ymax=1, color='gray', alpha=0.6, linewidth=1.2, linestyles='dashdot')
    ax.vlines(x=freqs_electrodos, ymin=y3[0], ymax=y1[0], color='darkgray', alpha=0.95, linewidth=2, linestyles='dashdot') #Igual pero más corta y marcada

    #cambiar esto a Title
    ax.set_title('Electrodos seleccionados con el esquema de reasignación', color = 'k', style = 'italic', pad = 20, fontdict={'size':32, 'font': 'Times New Roman'}, )

    #Texto para cada marquita de los elec
    elec_labels = ['$e_{' +str(i+1) +'}$ ' for i in range(elec_array.Nchann) ]
    act_labels  = ['$e_{' +str(i+1) +'}$ ' for i in elecs_activados ]

    #############################################3

    tuple1 = (freqs_electrodos, 'navy', 'Frecuencias de electrodos', elec_labels, y1, 'x')
    tuple2 = (freqs_nota, 'maroon', 'Armónicos de la nota objetivo', _, y2, 's')
    tuple3 = (freqs_activadas, 'darkgreen', 'Electrodos seleccionados', act_labels, y3, 'o')

    #Grafico las tres lineas de puntitos
    for freq_color_label_ticklabels_y_forma in [tuple2 , tuple3]:

        (freqs, color, label, ticklabels, y, formita) = freq_color_label_ticklabels_y_forma #unpack

        ax.scatter(y=y, x=freqs, s=600, color=color, alpha=0.9, label = label ,marker = formita) #Los puntitos
        ax.grid(alpha = 0.7)

    #Para los electodos apagados en la linea 3, repito más transparente la última linea y le agrego cositas:
    ax.scatter(y=[y3[0] for f in freqs_electrodos], x=freqs_electrodos, s=125, color='navy', alpha=0.45) #Los puntitos chiquitos y transparentes
    ax.scatter(y=y3, x=freqs_activadas, s=125, color='yellow', alpha=0.9, marker = '2') 

    #Eje de arriba: Toda la info
    (freqs, color, label, ticklabels, y, formita) = tuple1

    secax = ax.secondary_xaxis('top')  
    secax_freqs = ax.secondary_xaxis('top')  
    freq_labels = [f'${round(freq)}$ ' for freq in freqs ]
    secax.set_xticks(freqs[::step], labels = ticklabels[::step])
    secax.xaxis.set_tick_params(pad = 30, labelcolor = color, labelsize = 20, rotation = 30)
    secax_freqs.set_xticks(freqs[::step], labels = freq_labels[::step])
    secax_freqs.xaxis.set_tick_params(pad = 7, labelcolor = color, labelsize= 20)
    for i in range(1,len(freqs)//step+1):
        secax.get_xticklabels()[-i].set_rotation_mode('anchor')

    #Eje de abajo: Solo electrodos activados
    (freqs, color, label, ticklabels, y, formita) = tuple3
    secax = ax.secondary_xaxis('bottom')  
    secax.set_xticks(freqs, labels = ticklabels)
    secax.xaxis.set_tick_params(pad = 5, labelcolor = color, labelsize = 30, rotation = 30)
    for i in range(1,len(freqs)//step+1):
        secax.get_xticklabels()[-i].set_rotation_mode('anchor')


    #Freqs de los armónicos
    for i, freq in enumerate(nota.parciales):
        plt.text(freq, 0.5, int(freq), fontdict={'size':20, 'font': 'Times New Roman', 'color':'maroon'})
        plt.text(freq*0.975, 0.44, int(i+1), fontdict={'size':20, 'font': 'Times New Roman', 'color':'yellow'})


    #ax.set_xticks([], minor=True)


    ######################################

    # Title, Label, Ticks and Ylim
    #ax.set_yticks(df.index)
    #ax.set_yticklabels(df.manufacturer.str.title(), fontdict={'horizontalalignment': 'right'})
    plt.legend(framealpha=1, shadow=True,  loc = 'upper right',  prop={'size': 24})
    plt.show()

## Tablas 2 y 3: Con Cochlear 

In [ ]:
fund = 870
amp_armonicos = [1,1, 1,1,1, 1]
dur_seg = 2
nota = Nota(fund, amp_armonicos, dur_seg, [])
elec_array = GENERIC_COCHLEAR3

nChann = 4
max_di = 30

step = 2 #para la info de los elecs

grafico1(GENERIC_COCHLEAR3, nota, nChann, max_di, step)


Diente de sierra (6 canales)

In [ ]:
fund = 870
amp_armonicos = [1,1/2, 1/3,1/4,1/5, 1/6, 1/7, 1/8]
dur_seg = 2
nota = Nota(fund, amp_armonicos, dur_seg, [])
elec_array = GENERIC_COCHLEAR3

nChann = 6
max_di = 30

step = 2 #para la info de los elecs

grafico1(GENERIC_COCHLEAR3, nota, nChann, max_di, step)


Diente de sierra (4 canales)

In [ ]:
fund = 870
amp_armonicos = [1,1/2, 1/3,1/4,1/5, 1/6, 1/7, 1/8]
dur_seg = 2
nota = Nota(fund, amp_armonicos, dur_seg, [])
elec_array = GENERIC_COCHLEAR3

nChann = 4
max_di = 30

step = 2 #para la info de los elecs

grafico1(GENERIC_COCHLEAR3, nota, nChann, max_di, step)


4J por debajo

In [ ]:
fund = 870 *3/4 #4J por debajo
amp_armonicos = [1,1/2, 1/3,1/4,1/5, 1/6, 1/7, 1/8]
dur_seg = 2
nota = Nota(fund, amp_armonicos, dur_seg, [])
elec_array = GENERIC_COCHLEAR3

nChann = 4
max_di = 30

step = 2 #para la info de los elecs

grafico1(GENERIC_COCHLEAR3, nota, nChann, max_di, step)


5J por arriba

In [ ]:
fund = 870 *3/2 #5J por arriba
amp_armonicos = [1,1/2, 1/3,1/4,1/5, 1/6, 1/7, 1/8]
dur_seg = 2
nota = Nota(fund, amp_armonicos, dur_seg, [])
elec_array = GENERIC_COCHLEAR3

nChann = 6
max_di = 30

step = 2 #para la info de los elecs

grafico1(GENERIC_COCHLEAR3, nota, nChann, max_di, step, xmin = 1000, xmax = 13000)


## Resultado
Rinse & repeat para una escala mayor (veo solo los resultados numéricos para reunir en una tabla el mayor error)

In [ ]:
i = 1
for I in [1, 9/8, 5/4, 4/3, 3/2, 8/5, 5/3, 2]:

    fund = 870 *2/3 *I
    amp_armonicos = [1,1/2, 1/3,1/4,1/5, 1/6, 1/7, 1/8]
    dur_seg = 2
    nota = Nota(fund, amp_armonicos, dur_seg, [])
    elec_array = GENERIC_COCHLEAR3

    nChann = 6
    max_di = 100

    step = 2 #para la info de los elecs
    print(fund, i)
    print()
    grafico1(GENERIC_COCHLEAR3, nota, nChann, max_di, step, xmin = fund*0.9, xmax = 13000)
    print('---------------')
    i += 1

Resultado: Para Cochlear se pudo representar una escala mayor con error individual por armónico de hasta 100cents y con entre 5 y 6 canales x vez

--------------------------------

In [ ]:
i = 1
for I in [1, 9/8, 5/4, 4/3, 3/2, 8/5, 5/3, 2]:

    fund = 870*3/4 *I
    amp_armonicos = [1,1/2, 1/3,1/4,1/5, 1/6, 1/7, 1/8]
    dur_seg = 2
    nota = Nota(fund, amp_armonicos, dur_seg, [])
    elec_array = GENERIC_COCHLEAR3

    nChann = 6
    max_di = 100

    step = 2 #para la info de los elecs
    print(fund, i)
    print()
    grafico1(GENERIC_MEDEL3, nota, nChann, max_di, step, xmin = fund*0.9, xmax = 13000)
    print('---------------')
    i += 1

Resultado: No tan bueno para Medel

----------

Gráfico 3 - Junto la comparación de electrodos de Medel y Cochlear en uno solo, que quedo bueno! (intento luego)
----

In [ ]:
fund = 870
amp_armonicos = [1,1, 1,1,1, 1]
dur_seg = 2
nota = Nota(fund, amp_armonicos, dur_seg, [])

nChann = 4
max_di = 300

step = 2

y1, y2, y3 = 0.4, 0.45, 0.55

mejorPatronMedel = encontrarMejorMatch2(nota, GENERIC_MEDEL3.freqsElectrodos, nChann, max_di)
mejorPatronCochlear = encontrarMejorMatch2(nota, GENERIC_COCHLEAR3.freqsElectrodos, nChann, max_di)

elecs_activados_Medel, freqs_Medel = [], []
elecs_activados_Cochlear, freqs_Cochlear = [], []

for i, elec in enumerate(mejorPatronMedel.elecs):
    if elec.on: 
        elecs_activados_Medel.append(i)
        freqs_Medel.append(elec.freq)
for i, elec in enumerate(mejorPatronCochlear.elecs):
    if elec.on: 
        elecs_activados_Cochlear.append(i)
        freqs_Cochlear.append(elec.freq)

#####################################
#Electrodos: Arriba
#Medio:  Nota y sus armónicos
#Abajo: Electrodos activados


###########

import matplotlib.ticker as ticker
xmin = 500
xmax = 9000

fig, ax = plt.subplots(figsize=(16,6), dpi= 130)
ax.set_xscale('log')
ax.set_xlabel('Frecuencia (Hz)', labelpad = 35,fontdict={'size':25} )
ax.xaxis.set_tick_params(pad = 7)
ax.set_xlim(xmin , xmax)
ax.set_ylim(0.1 , 0.6)
ax.set_yticks([])
ax.set_xticks([])
ax.hlines(y=[y1,y2,y3], xmin=xmin, xmax=xmax, color='gray', alpha=0.85, linewidth=1, linestyles='dashdot')

ax.vlines(x=nota.parciales, ymin=0, ymax=1, color='gray', alpha=0.6, linewidth=1.2, linestyles='dashdot')
ax.vlines(x=nota.parciales, ymin=y3, ymax=y1, color='darkgray', alpha=0.95, linewidth=2, linestyles='dashdot') #Igual pero más corta y marcada

#cambiar esto a Title
ax.set_title('Electrodos seleccionados con el esquema de reasignación', color = 'k', style = 'italic', pad = 20, fontdict={'size':32, 'font': 'Times New Roman'}, )

#Texto para cada marquita de los elec
medel_labels = ['$e_{' +str(i+1) +'}$ ' for i in elecs_activados_Medel ]
cochlear_labels  = ['$e_{' +str(i+1) +'}$ ' for i in elecs_activados_Cochlear]

#############################################3

tuple1 = (freqs_Medel, 'navy', 'Flex28 (N=12)',medel_labels , 0.6, 'x')
tuple2 = (nota.parciales, 'maroon', 'Armónicos de la nota objetivo', _, 0.45, 's')
tuple3 = (freqs_Cochlear, 'darkgreen', 'Contour Advance (N=22)',cochlear_labels , 0.3, '.')


#Grafico las tres lineas de puntitos
for freq_color_label_ticklabels_y_forma in [tuple1, tuple2 , tuple3]:

    (freqs, color, label, ticklabels, y0, formita) = freq_color_label_ticklabels_y_forma #unpack
    
    y = [y0 for _ in freqs]

    ax.scatter(y=y, x=freqs, s=600, color=color, alpha=0.9, label = label ,marker = formita) #Los puntitos
    ax.grid(alpha = 0.7)

#Para los electodos apagados en la linea 1 y 3, agrego bolitas transparentes:
ax.scatter(y=[y3 for _ in range(GENERIC_MEDEL3.Nchann)], x=GENERIC_MEDEL3.freqsElectrodos, s=125, color='navy', alpha=0.45) #Los puntitos chiquitos y transparentes
ax.scatter(y=[y3 for _ in elecs_activados_Medel], x=elecs_activados_Medel, s=125, color='yellow', alpha=0.9, marker = '2') 

#Eje de arriba: Toda la info
for pair in [('top', tuple1), ('bottom', tuple3)]:
    pos, tuplex = pair[0], pair[1] 
    (freqs, color, label, ticklabels, y, formita) = tuplex

    secax = ax.secondary_xaxis(pos)  
    secax_freqs = ax.secondary_xaxis(pos)  
    freq_labels = [f'${round(freq)}$ ' for freq in freqs ]
    secax.set_xticks(freqs[::step], labels = ticklabels[::step])
    secax.xaxis.set_tick_params(pad = 30, labelcolor = color, labelsize = 20, rotation = 30)
    secax_freqs.set_xticks(freqs[::step], labels = freq_labels[::step])
    secax_freqs.xaxis.set_tick_params(pad = 7, labelcolor = color, labelsize= 20)
    for i in range(1,len(freqs)//step+1):
        secax.get_xticklabels()[-i].set_rotation_mode('anchor')


#Freqs de los armónicos (medio)
for i, freq in enumerate(nota.parciales):
    plt.text(freq, y2*0.9, int(freq), fontdict={'size':20, 'font': 'Times New Roman', 'color':'maroon'})
    plt.text(freq*0.975, y2*0.98, int(i+1), fontdict={'size':20, 'font': 'Times New Roman', 'color':'yellow'})


#ax.set_xticks([], minor=True)


######################################

# Title, Label, Ticks and Ylim
#ax.set_yticks(df.index)
#ax.set_yticklabels(df.manufacturer.str.title(), fontdict={'horizontalalignment': 'right'})
plt.legend(framealpha=1, shadow=True,  loc = 'lower left',  prop={'size': 24})
plt.show()

---------------------

Y ahora va de vuelta sin usar reasignación

In [ ]:
sig = nota.sintetizar(elec_array.SR)

print('Nota original' )
display(Audio(sig, rate=IMPLANT_SR))
plot_stft_señalsola(sig, elec_array.freqsAnalisis)

Hago un gráfico con el espectrograma de:
-Nota original
-Señal producida normalmente por el implante
-Señal producida con la reasignación

In [ ]:
from scipy.fft import rfft, rfftfreq

DISPERTION = 0.1

sig = nota.sintetizar(OUT_SR)
out_normal = elec_array.CIS(sig, OUT_SR)*200
out_reasignada = elec_array.CIS(elec_array.generarSenialMejorMatchNota(nota, 4, 50), OUT_SR)*100

#Generales
sr = OUT_SR
f_from = 200
f_to = 10000
range_dB = 50 
norm = 1
logx = True

T = 1/sr #periodo
N = len(sig)
t = np.arange(0, N*T, T) # Sampling times

# FFT and bin centers
f = rfftfreq(N, T)
#df : sr/N  --> f(i) = i* sr/N --> i = f*N/sr  
if(f_to == 0): fto = sr/2
ifrom = round(f_from*N/sr)
ito = round(f_to*N/sr)

# Plot spectrum
fig, ax = plt.subplots(dpi = 200)
if(logx): ax.set_xscale('log')


#Plot señales
for sigcolor_pair in [(sig,'black') , (out_normal, 'red'), (out_reasignada, 'blue')]:
 
  (x, color) = sigcolor_pair #unpack

  y = rfft(x)
  y = np.abs(y)*2/N
  y /= norm
  #y = 20*np.log10(np.abs(y)) #dB
  y_db = 20*np.log10(y)

  ax.plot(f[ifrom:ito], y_db[ifrom:ito], color = color, linewidth = 0.5) #eje lineal, y logaritmico
  y_max =  max(y) +5
  y_min = max(y) -range_dB
  #ax.set_xlim(f_from, f_to)
  ax.set_ylim(y_min , y_max)

ax.legend((f'Señal de entrada', f'Simulación (sin reasignación)', f'Simulación (con reasignación)'), loc="upper right")

#Graficar lineas verticales para los electrodos
showElecs = True
if(showElecs):
  color = 'blue'
  secax = ax.secondary_xaxis('top') #eje secundario arriba
  secax.set_xlabel('(Frecuencia característica de los electrodos)', color = 'dimgray', style = 'italic')
      

  #Creo marquitas para los elec
  elec_labels = ['$e_{' +str(i) +'}$ ' for i in range(elec_array.Nchann) ]

  #Agrego marquitas
  step =1 
  secax.set_xticks(elec_array.freqsElectrodos[::step], labels = elec_labels[::step])
  
  #Les pongo color
  for i in range(1,elec_array.Nchann//step+1):
    secax.get_xticklabels()[-i].set_color('limegreen')
    secax.get_xticklabels()[-i].set_rotation(45)
    secax.get_xticklabels()[-i].set_rotation_mode('anchor')


  ax.vlines(elec_array.freqsElectrodos,y_min , y_max , colors=color, linewidth = 1.3, linestyle = (0, (1,5)), alpha = 0.5)


plt.xlabel('Frecuencia (Hz)')
plt.ylabel('Amplitud (dB)')  
plt.grid(alpha  = 0.8)


Hallo las frecuencias con mayor energía de un espectrograma. Esto no lo uso más porque no me anduvo en otros casos

In [ ]:
fund = 870
amp_armonicos = [1,1/2, 1/3,1/4,1/5, 1/6,1/7,1/8]
dur_seg = 2
nota = Nota(fund, amp_armonicos, dur_seg, [])

sig = nota.sintetizar(OUT_SR)

y= rfft(sig)
y = np.abs(y)*2/N
#store list in tmp to retrieve index

#lista de indices  según energía, de mayor a menor
def indices_ordenados(lista):
    out = sorted(range(len(lista)), key=lambda x: lista[x])
    out.reverse()
    return out

print('Frecuencias de la nota sintetizada:', nota.parciales)
print('Frecuencias con mayor energia en FFT:' , f[indices_ordenados(y)][0:5])


---------------

Repito el gráfico de los puntos para comparar el esquema sin reasignación vs con reasignación

In [ ]:
#Defino la nota que quiero

fund = 870
amp_armonicos = [1,0, 2,0,1/5, 1.5]
dur_seg = 2 #larga, para afinar bien con picos únicos la fft
nota = Nota(fund, amp_armonicos, dur_seg, [])

DISPERTION = 0 #Uso sinusoides para sintetizar la salida, así queda más fácil encontrar los picos
sig = nota.sintetizar(OUT_SR)
out_normal = elec_array.CIS(sig, OUT_SR)
out_reasignada = elec_array.CIS(elec_array.generarSenialMejorMatchNota(nota, 4, 50), OUT_SR)

T = 1/OUT_SR #periodo
N = len(sig)
t = np.arange(0, N*T, T) # Sampling time

# FFT and bin centers
freqs_fft = rfftfreq(N, T)
y= rfft(out_normal)
y = np.abs(y)*2/N
y /= max(y) #norm


In [ ]:
print(np.round(elec_array.freqsElectrodos))
print()

from scipy.signal import find_peaks
peaks, _ = find_peaks(y, distance = 100, prominence= 0.05) 

print(freqs_fft[peaks])
print()
print('Frecuencias con mayor energia en FFT:' , freqs_fft[peaks[indices_ordenados(peaks)]][0:5])

plt.plot(freqs_fft, y) #Versión suavizada
plt.plot(freqs_fft[peaks], y[peaks], "x", color = 'red')

In [ ]:

#####################################
#Arriba: Electrodos activados (sin reasignación)
#Medio:  Nota y sus armónicos
#Abajo: Electrodos activados (con reasignación)

#Para calcular las frecuencias máximas, voy a usar "find_peaks", sobre la fft

###########

import matplotlib.ticker as ticker
xmin = 220
xmax = 13000

fig, ax = plt.subplots(figsize=(16,6), dpi= 130)
ax.set_xscale('log')
ax.set_xlabel('Frecuencia (Hz)', labelpad = 35,fontdict={'size':25} )
ax.xaxis.set_tick_params(pad = 7)
ax.set_xlim(xmin , xmax)
ax.set_ylim(0.1 , 1)
ax.set_yticks([])
ax.set_xticks([])
ax.hlines(y=[0.6,0.45, 0.3], xmin=xmin, xmax=xmax, color='gray', alpha=0.85, linewidth=1, linestyles='dashdot')
ax.vlines(x=elec_array.freqsElectrodos, ymin=0, ymax=1, color='gray', alpha=0.6, linewidth=1.2, linestyles='dashdot')
ax.vlines(x=elec_array.freqsElectrodos, ymin=0.3, ymax=0.6, color='darkgray', alpha=0.95, linewidth=2, linestyles='dashdot') #Igual pero más corta y marcada

#cambiar esto a Title
ax.set_title('Electrodos estimulados', color = 'k', style = 'italic', pad = 20, fontdict={'size':32, 'font': 'Times New Roman'}, )

#Texto para cada marquita de los elec
elec_labels = ['$e_{' +str(i+1) +'}$ ' for i in range(elec_array.Nchann) ]

#############################################3
step = 1

def espectroNormalizado(s):
    Fs = rfft(s)
    Fs = np.abs(Fs)*2/len(s)
    Fs /= max(Fs) #norm
    return Fs #Cambio el primer componente



F1 = espectroNormalizado(sig)
F2 = espectroNormalizado(out_normal)
F3 = espectroNormalizado(out_reasignada)
tuple1 = (F1, 'navy', 'Activaciones (sin reasignación)', elec_labels, 0.6)
tuple2 = (F2, 'maroon', 'Componentes de la nota objetivo', _, 0.45)
tuple3 = (F3, 'darkgreen', 'Activaciones (con reasignación)', act_labels, 0.3)

#Grafico las tres lineas de puntitos
#El tamaño de los puntos es proporcional a la amplitud del espectro
for Fs_color_label_ticklabels_y in [tuple1, tuple2 , tuple3]:
   
    (Fs, color, label, ticklabels, y0) = Fs_color_label_ticklabels_y #unpack
    
    #Encuentro los picos
    peaks, _ = find_peaks(Fs, distance = 100, prominence= 0.05) 
    #Me quedo con los n más grandes
    max_freqs = freqs_fft[peaks] 


    #Grafico
    y = [ y0 for f in max_freqs]
    tamaños = np.array(Fs[peaks]) #De los circulitos
    ax.scatter(y=y, x=max_freqs, s=300*tamaños, color=color, alpha=0.85, label = label) #Los puntitos
    ax.grid(alpha = 0.7)

#Para los electodos apagados en la linea 3, repito más transparente la última linea y le agrego cositas:
ax.scatter(y=[0.3 for f in elec_array.freqsElectrodos], x=elec_array.freqsElectrodos, s=125, color='navy', alpha=0.45) #Los puntitos chiquitos y transparentes

#Eje de arriba y de abajo
for pair in  [(tuple1, 'top') , (tuple3, 'bottom')]:

    Fs_color_label_ticklabels_y, pos = pair[0], pair[1]
    (Fs, color, label, ticklabels, y) = Fs_color_label_ticklabels_y

    #Calculo esto de vuelta
    peaks, _ = find_peaks(Fs, distance = 100, prominence= 0.05) 
    max_freqs = freqs_fft[peaks] 

    secax = ax.secondary_xaxis(pos) #eje secundario (texto)    
    secax_freqs = ax.secondary_xaxis(pos) #eje secundario (numeros)    
    print(max_freqs)

    freq_labels = [f'${round(freq)}$ ' for freq in max_freqs ]

    secax.set_xticks(max_freqs[::step], labels = ticklabels[::step])
    secax.xaxis.set_tick_params(pad = 25, labelcolor = color, labelsize = 20, rotation = 30)
    secax_freqs.set_xticks(max_freqs, labels = freq_labels[::step])
    secax_freqs.xaxis.set_tick_params(pad = 7, labelcolor = color, labelsize= 20)
    for i in range(1,len(max_freqs)//step+1):
        secax.get_xticklabels()[-i].set_rotation_mode('anchor')

#ax.set_xticks([], minor=True)


######################################

# Title, Label, Ticks and Ylim
#ax.set_yticks(df.index)
#ax.set_yticklabels(df.manufacturer.str.title(), fontdict={'horizontalalignment': 'right'})
plt.legend(framealpha=1, shadow=True,  loc = 'upper right',  prop={'size': 20})
plt.show()

Yyyy me doy cuenta después de todo esto que en realidad era mucho más fácil: En vez de buscar los picos del espectro, voy a ver cuanto dio la fft en la freq de los electrodos... ayayay era así de simple

## Grafico 2 - Comparo electrodos activados Con y Sin mi programa

In [ ]:
#Saca la fft completa y despues mira algunos indices
def espectroElectrodos(s, norm = True):
    Fs = rfft(s)
    Fs = np.abs(Fs)*2/len(s)
    if(norm): Fs /= max(Fs) 

    indices_electrodos = np.int32(np.round(np.array(fes) * dur_seg))
    Fs = Fs[indices_electrodos]
    return Fs

#Saco solo la dft de las freqs que me intersan. Más robusto por los indices tambien
def espectroElectrodos2(s, sr, norm = True):
    
    dur = len(s)/sr
    Fs = []
    for f in elec_array.freqsElectrodos:
        sinsig = puretone(f,dur, sr, phase = 0)
        cossig = puretone(f,dur,sr, phase = 90)

        A = np.dot(sinsig, s)**2 + np.dot(cossig, s)**2
        A = np.sqrt(A)
        Fs.append(A)
        
    if(norm): Fs/=max(Fs)
    return Fs

In [ ]:
#Defino la nota que quiero
fund = 870
amp_armonicos = [1,0.5, 2,0.1,1/5, 1.5]
dur_seg = 2 
nota = Nota(fund, amp_armonicos, dur_seg, [])

DISPERTION = 0.01 #Uso bandas de ruido finas para la salida, porque agarro los picos por los indices aproximados
ORDEN = 12
CHANNEL_INTERACTION_INDEX = 0
elec_array = GENERIC_MEDEL3

sig = nota.sintetizar(OUT_SR)
out_normal = elec_array.CIS(sig, OUT_SR)
out_reasignada = elec_array.CIS(elec_array.generarSenialMejorMatchNota(nota, 4, 50), OUT_SR)

In [ ]:
#####################################
#Arriba: Electrodos activados (sin reasignación)
#Medio:  Nota y sus armónicos
#Abajo: Electrodos activados (con reasignación)

#Para calcular las frecuencias máximas, voy a usar "find_peaks", sobre la fft

###########

import matplotlib.ticker as ticker
xmin = 450
xmax = 6500

fes = elec_array.freqsElectrodos

(y1, y2, y3) = (0.58,0.5, 0.42)

#Generalidades gráfico
fig, ax = plt.subplots(figsize=(16,6), dpi= 130)
ax.set_xscale('log')
ax.set_xlabel('Frecuencia (Hz)', labelpad = 28,fontdict={'size':30} )
ax.xaxis.set_tick_params(pad = 7)
ax.set_xlim(xmin , xmax)
ax.set_ylim(0.35 , 0.8)
ax.set_yticks([])
ax.set_xticks([])
ax.hlines(y=[y1, y2, y3], xmin=xmin, xmax=xmax, color='gray', alpha=0.85, linewidth=1, linestyles='dashdot')
ax.vlines(x=fes, ymin=0, ymax=1, color='gray', alpha=0.6, linewidth=1.8, linestyles='dashdot')
ax.vlines(x=nota.parciales, ymin=0, ymax=1, color='maroon', alpha=0.5, linewidth=1, linestyles='dashed')
#ax.vlines(x=elec_array.freqsElectrodos, ymin=0.3, ymax=0.6, color='darkgray', alpha=0.95, linewidth=2, linestyles='dashdot') #Igual pero más corta y marcada

#Title
ax.set_title('Simulación de electrodos estimulados con el esquema de reasignación', color = 'k', style = 'italic', pad = 20, fontdict={'size':32, 'font': 'Times New Roman'}, )

#Texto para cada marquita de los elec
elec_labels = ['$e_{' +str(i+1) +'}$ ' for i in range(elec_array.Nchann) ]

#############################################3
step = 1


F2 = espectroElectrodos2(out_normal, OUT_SR)
F3 = espectroElectrodos2(out_reasignada, OUT_SR)*3
tuple_nota = (nota.parciales,nota.amps, 'maroon', 'Componentes de la nota objetivo' , _, y2,'s')
tuple_normal = (fes, F2, 'navy', 'Activaciones (sin reasignación)' , elec_labels, y1, 'D')
tuple_realoc = (fes, F3, 'darkgreen', 'Activaciones (con reasignación)', elec_labels, y3, 'P')

#Grafico las tres lineas de puntitos
#El tamaño de los puntos es proporcional a la amplitud del espectro
for frecuencias_amplitudes_color_label_ticklabels_y_marker in [tuple_normal ,tuple_nota, tuple_realoc]:
   
    (freqs, amps, color, label, ticklabels, y0, forma) = frecuencias_amplitudes_color_label_ticklabels_y_marker #unpack
    
    #Grafico
    y = [ y0 for f in freqs]
    ax.scatter(y=y, x=freqs, s=650*np.array(amps)**0.5, color=color, alpha=0.85, label = label, marker=forma) #Los puntitos
    ax.grid(alpha = 0.7)

#Para los electodos apagados en la linea 3, repito más transparente la última linea y le agrego cositas:
#ax.scatter(y=[0.3 for f in elec_array.freqsElectrodos], x=elec_array.freqsElectrodos, s=125, color='navy', alpha=0.45) #Los puntitos chiquitos y transparentes

#Eje de arriba: e1, e2...
secax = ax.secondary_xaxis('top') #eje secundario (texto)    
secax.set_xticks(fes[::step], labels = elec_labels[::step])
secax.xaxis.set_tick_params(pad = 8, labelcolor = 'navy', labelsize = 30, rotation = 30)
for i in range(1,len(fes)//step+1):
    secax.get_xticklabels()[-i].set_rotation_mode('anchor')

#Eje de arriba: Freqs electrodos
secax_freqs = ax.secondary_xaxis('top') #eje secundario (numeros)    
freq_labels = [f'${round(freq)}$ ' for freq in fes ]
secax_freqs.set_xticks(fes, labels = freq_labels[::step])
secax_freqs.xaxis.set_tick_params(pad = 30, labelcolor = 'navy', labelsize= 20)


#Eje de abajo: freqs nota
secax_freqs_nota = ax.secondary_xaxis('bottom') #eje secundario (numeros)    
freq_labels = [f'${round(freq)}$ ' for freq in nota.parciales ]
secax_freqs_nota.set_xticks(nota.parciales, labels = freq_labels[::step])
secax_freqs_nota.xaxis.set_tick_params(pad = 7, labelcolor = 'maroon', labelsize= 20)

#ax.set_xticks([], minor=True)

#Freqs de los armónicos
for i, freq in enumerate(nota.parciales):
    plt.text(freq*0.975, 0.49, str(int(i+1))+'°', fontdict={'size':20, 'font': 'Times New Roman', 'color':'yellow'})


######################################

# Title, Label, Ticks and Ylim
#ax.set_yticks(df.index)
#ax.set_yticklabels(df.manufacturer.str.title(), fontdict={'horizontalalignment': 'right'})
plt.legend(framealpha=1, shadow=True,  loc = 'upper left',  prop={'size': 24})
plt.show()

In [ ]:
plt.plot(freqs_fft[0:4000], espectroNormalizado(out_reasignada)[0:4000])
plt.plot(freqs_fft[0:4000], espectroNormalizado(out_normal)[0:4000])
plt.plot(nota.parciales, nota.amps)

In [ ]:
nChann = 4
max_di = 50
#amp_armonicos = [1,0.5, 2,0,1/5, 1.5]
mejorPatron = encontrarMejorMatch2(nota, elec_array.freqsElectrodos, nChann, max_di)
mejorPatron.print()

## Resultado 2 - Mejor Trasposicion de una escala

Med-El

In [ ]:

fund = 870
amp_armonicos = [1/n for n in range(1,9)]
I = [1, 9/8, 5/4, 4/3, 3/2, 8/5, 5/3, 2]
dur_seg = 2 

notas = [Nota(fund*i, amp_armonicos, dur_seg, []) for i in I]  #Melodía: Escala Mayor

nChann = 4
max_di = 50

Melodias_Medel = GENERIC_MEDEL3.buscarMejorTrasposicionMelodia(Melodia(notas), nChann, max_di) #lista de mejores patrones para cada trasposicion de la melodia

In [ ]:
for melodia in Melodias_Medel:
    print(np.round(melodia.notasExtremas()), np.round(melodia.score()))

In [ ]:
grafico1(GENERIC_MEDEL3, Melodias_Medel[0].notas[7], nChann, max_di, step=1, xmin= 500, xmax = 9000)

Las dos mejores fueron del registro [1280. 2560.] (#14) y [ 636. 1271.] (#8). Pero la #8 no es posible con la restriccion de max_di. En cambio, la #9, que tiene más puntaje, sí

In [ ]:
Melodias_Medel[-3].mejorSecuenciaPatrones[1].print()

In [ ]:
fund = 870
amp_armonicos = [1/n for n in range(1,9)]
I = [1, 9/8, 5/4, 4/3, 3/2, 8/5, 5/3, 2]
dur_seg = 2 

notas = [Nota(fund*i, amp_armonicos, dur_seg, []) for i in I]  #Melodía: Escala Mayor

nChann = 4
max_di = 50

Melodias_Cochlear = GENERIC_COCHLEAR3.buscarMejorTrasposicionMelodia(Melodia(notas), nChann, max_di) #lista de mejores patrones para cada trasposicion de la melodia

In [ ]:
for melodia in Melodias_Cochlear[20:]:
    print(np.round(melodia.notasExtremas()), np.round(melodia.score()))

La mejor es la #10 (803, 1605), pero es una poronga, no representa nadaaaa

In [ ]:
grafico1(GENERIC_COCHLEAR3, Melodias_Cochlear[24].notas[4], nChann, max_di, step=1, xmin= 500, xmax = 9000)
Melodias_Cochlear[1].mejorSecuenciaPatrones[5].print()

# Prueba Preliminar 2b
Dos armónicos, en relación de 8v+5ta (2°H y 3°H)

In [ ]:
ORDEN = 20
nChann = 2
max_di = 200
fund = 870
amp_armonicos = [0,1,1]


test_suite_Nota(fund, amp_armonicos, GENERIC_MEDEL3, nChann,max_di)

# Prueba Preliminar 3: Nota completa sintetizada
Este es un caso interesante y demuestra que los armonicos superiores se pueden priorizar por sobre los mas bajos si uno no tiene cuidado
En este caso, el intervalo entre la fundamental y el electrodo 4 es bajo, pero hay un armonico superior que resulta tener intervalo aun mas bajo y el algoritmo lo prioriza. Esto se nota más cuando hay pocos canales.

Que el algoritmo priorice los armonicos tan superiores es malo, asi que voy a corregir el algoritmo hasta que masomenos elija la fundamental

In [ ]:
def scoreRelacion(f, dist, amp, armonico_i):
    return (dist * amp)**2

In [ ]:
fund = 870
amp_armonicos = saw
dur_seg = 3

print('Intervalo entre fundamental y electrodo 4: '+ str(cents(fund,GENERIC_MEDEL3.freqsElectrodos[3] )))
print('Intervalo entre armonico 5  y electrodo 9: '+ str(cents(fund*5,GENERIC_MEDEL3.freqsElectrodos[8] )))
print('Intervalo entre armonico 9  y electrodo 11: '+ str(cents(fund*9,GENERIC_MEDEL3.freqsElectrodos[10] )))

print()

print('Error para fundamental y electrodo 4: '  + str(scoreRelacion(fund, -45, 1, 1)))
print('Error para armonico 5 y electrodo 9:  '  + str(scoreRelacion(fund*5, 25, 1/5, 5)))

In [ ]:
ORDEN = 20
nChann = 2
max_di = 200

test_suite_Nota(fund, amp_armonicos, GENERIC_MEDEL3, nChann,max_di)

## 3B: Pruebo cambios en la funcion hasta que no pase

In [ ]:
def scoreRelacion(f, dist, amp, armonico_i):
    return (dist/50)**2 * (2**armonico_i)

In [ ]:
fund = 870
amp_armonicos = saw
dur_seg = 3

print('Intervalo entre fundamental y electrodo 4: '+ str(cents(fund,GENERIC_MEDEL3.freqsElectrodos[3] )))
print('Intervalo entre armonico 5  y electrodo 9: '+ str(cents(fund*5,GENERIC_MEDEL3.freqsElectrodos[8] )))
print('Intervalo entre armonico 9  y electrodo 11: '+ str(cents(fund*9,GENERIC_MEDEL3.freqsElectrodos[10] )))

print()

print('Error para fundamental y electrodo 4: '  + str(scoreRelacion(fund, -45, 1, 1)))
print('Error para armonico 5 y electrodo 9:  '  + str(scoreRelacion(fund*5, 25, 1/5, 5)))
print('Error para armonico 9 y electrodo 11:  '  + str(scoreRelacion(fund*9, 13, 1/9, 9)))

Con esto gana la fundamental por sobre el armonico 5 y el 9

In [ ]:
ORDEN = 20
amp_armonicos = saw
nChann = 2
max_di = 200

test_suite_Nota(fund, amp_armonicos, GENERIC_MEDEL3, nChann,max_di)

Ahora, si cambio la forma de onda a una que tenga solo armonicos impares...

In [ ]:
ORDEN = 20
amp_armonicos = sq
nChann = 2
max_di = 200

test_suite_Nota(fund, amp_armonicos, GENERIC_MEDEL3, nChann,max_di)

Cambia la relación! Al faltar el armónico 2, que hubiera sido elegido con la diente de sierra.
Por eso es importante usar una buena forma de onda base para darle al algoritmo que estoy diseñando

# Prueba preliminar 4: Pequeña melodía

Acá vemos un problema claro: Con pocos canales, nada me garantiza que en una melodía una nota preserve sus armónicos bajos
Con dos canales:

In [ ]:
def test_suite_Melodia(funds, amps, durs, elec_array, nChann,max_di):
    MISMATCH = 0
    CHANNEL_INTERACTION_INDEX = 0
    DISPERTION = 0.001
    WIN_LEN_ACE = 1000
    MODE = CONTINOUS
    MAXIMA = 5
    amps_list = [amps]* len(funds) #lista duplicada n veces

    sig =  melody(funds, amps, durs, elec_array.SR)
    print('Señal original')
    plt.plot(sig[0:100])
    display(Audio(sig, rate= elec_array.SR))
    plot_stft_señalsola(sig, elec_array.freqsElectrodos)

    print('Señal sintetizada para que elicite esa altura')
    A = elec_array.generarSenialMejorMatchMelodia(funds, amps_list, nChann,max_di, durs)
    display(Audio(A, rate = elec_array.SR))
    #plot_stft_señalsola(A)
    plot_stft(A, elec_array, analisis = True, mini = True)

    print('Señal que daría el implante')
    out = elec_array.CIS(A)
    display(Audio(out, rate = OUT_SR))
    plot_stft(out,elec_array, analisis = False, mini = True)

In [ ]:
arpegioMayor = [69, 73, 76, 81]
intervalo5J = [69, 69+7]
freqs = [midi2freq(f) for f in arpegioMayor] #arpegio La Mayor
amps = saw
durs = [1,1,1,1]
melodia = melody(freqs, saw, durs, GENERIC_MEDEL3.SR)
test_suite_Melodia(freqs, amps, durs, GENERIC_MEDEL3, 2, 200)

Con 4 canales y permitiendo más margen de error en la diferencia en cents (200->300):

In [ ]:
test_suite_Melodia(freqs, amps, durs, GENERIC_MEDEL3, 4, 300)

Mirá que interesante: Las fundamentales mapeadas del 3er y 5to armónico quedan fijas...

-----------------------------------------------------------------------------

# Prueba Preliminar 5 - Mejor traspocisión

Busco que trasposición es la que menos distorsiona a una melodía para un implante dado

A modo de ejemplo, probemos el mismo arpegio de arriba una 8va abajo y una 8va arriba:

8va arriba: Canales demasiado separados

In [ ]:
arpegioMayor = [69, 73, 76, 81]
freqs = [midi2freq(n)*2 for n in arpegioMayor] #arpegio La Mayor
amps = saw
durs = [1,1,1,1]
melodia = melody(freqs, amps, durs, GENERIC_MEDEL3.SR)
test_suite_Melodia(freqs, amps, durs, GENERIC_MEDEL3, 3, 300)

8va abajo : No hay separacion suficiente entre los armonicos para cubrir electrodos

(con max d = 200 cents es estatico, con max d= 300 cents mejora pero se sigue perdiendo la direccionalidad)

In [ ]:
arpegioMayor = [69, 73, 76, 81]
freqs = [midi2freq(n)/2 for n in arpegioMayor] #arpegio La Mayor
melodia = melody(freqs, saw, durs, GENERIC_MEDEL3.SR)
test_suite_Melodia(freqs, amps, durs, GENERIC_MEDEL3, 3, 300)

Busco la trasposición óptima, que genere menos distorsión:

In [ ]:
amps_list = [amps]* len(freqs) #lista duplicada n veces

M = GENERIC_MEDEL3.buscarMejorTrasposicionMelodia(freqs, amps_list, 2, 300, durs) #lista de mejores patrones para cada trasposicion de la melodia

In [ ]:
scores = []
for patron_list in M:
    scores.append(sumaScores(patron_list))
print(scores)

In [ ]:
#Como se ve arriba, hay mucha variacion en los scores
#Parto de 250Hz y voy subiendo de a un semitono para probar.
#Escuchemos algunas trasposiciones que propone esto
fbase = 250 
ffinal = 3000
step = np.sqrt(16/15 * 2**(1/12)) #compromiso entre semitono justo y temperado . Con este step, hace 40 iteraciones

fundamentales_trnsp = np.array(freqs) * fbase/freqs[0] 
mejoresPatrones_list = []

for i in range(0,30,5):
    print('Trasposicion ' + str(i))
    fundamentales_trnsp = np.array(freqs) * (step**i) 
    sig = melody(fundamentales_trnsp, saw, durs, GENERIC_MEDEL3.SR)
    display(Audio(sig, rate = GENERIC_MEDEL3.SR))
    plot_stft_señalsola(sig, lineas = GENERIC_MEDEL3.freqsElectrodos)


Comparemos las primeras 5:

In [ ]:
for patron_list in M[0:5]:
    s = []
    for patron in patron_list:
        s += list(GENERIC_MEDEL3.generarSenialPatron(patron, 1, [1]))
    plot_stft(s,  GENERIC_MEDEL3, analisis = True, mini = True)
    display(Audio(s, rate = GENERIC_MEDEL3.SR))

Comparemos la mejor contra la peor

ANDA MAL, prque la mejor es horrible y la peor no es tan mala. Corregir

In [ ]:
mejor_i = scores.index(min(scores))
peor_i = scores.index(max(scores))
mejor = M[mejor_i]
peor = M[peor_i]
print('Mejor y peor: ' + str(mejor_i) +' ' + str(peor_i))

for patron_list in (M[mejor_i], M[peor_i]):
    s = []
    for patron in patron_list:
        s += list(GENERIC_MEDEL3.generarSenialPatron(patron, 1, [1]))
    plot_stft(s,  GENERIC_MEDEL3, analisis = True, mini = True)
    display(Audio(s, rate = GENERIC_MEDEL3.SR))


------------------------------------------------------

In [ ]:
filtered_bank = filter_bank_CIS(A, GENERIC_MEDEL3, ORDEN, analisis = True)

In [ ]:
ff = filtered_bank[3]
plt.plot(ff[1000:])
Audio(ff, rate = IMPLANT_SR)


In [ ]:
 filter_bank_CIS(wnoise, elec_array, ORDEN, analisis = False)

In [ ]:
#envelopes = [envelope_Cochlear(filtered_bank[i], ENVELOPE_FC, ENVELOPE_FULLWAVE) for i in range(N)] # CIS convencional
envelopes = [envelope_Hilbert(filtered_bank[i]) for i in range(12)] # "CIS+"

In [ ]:
k=5
#plt.plot(envelopes[k])
plt.plot(filtered_bank[k])

In [ ]:
wnoise = wnoise_sig(len(out))
filtered_noises = filter_bank_CIS(wnoise, GENERIC_MEDEL3, ORDEN, analisis = False)


In [ ]:
Audio(filtered_noises[4], rate = OUT_SR)

In [ ]:

s, phi = puretonePhase(550, 1, 44100, 0)
s2, _ = puretonePhase(700, 1, 44100, phi)
s += s2
plot_stft_señalsola(s)
display(Audio(s, rate=44100))

In [ ]:
x= np.exp([ 7*(-i/500) for i in range(500)])
plt.plot(x)

In [ ]:
s = puretone(500, 1, 44100)
s = adsre(150,300, 0.2, 200, s, 44100)

plt.plot(s)
display(Audio(s, rate = 44100))

# Prueba 6 - Comparación auditiva de la afincación de las notas originales vs las que elijo representar

In [ ]:
import import_ipynb
from CIS_6_Best_Match_v4 import * 
from IPython.display import Audio 


Ejemplo de importar una melodia midi, sintetizarla, y ver mas o menos que tanto se distorsionaria en la persona que la reciba

SIN TRASPOCISION 

emm eso de Trasposicion, solo fundamental, etc quedó viejo. Va la melo con la reasignacion completa v4 ya

## Solo la fundamental

In [ ]:
amplitudes_armonicos = np.array([1]*2)

m = Melodia.leerMidi(r"midis\Pablo Pasea.mid", amplitudes_armonicos)
out = m.sintetizar(44100, vibratoDepth = 0.2, vibratoFreq = 2)

print('Melo original - Pablo pasea')
Audio(out, rate = 44100)

Medel

In [ ]:
elec_array = GENERIC_MEDEL3
nChann = 3
max_di = 800
m.notas = m.notas[0:21] 

#patron =  encontrarMejorMatchMelodia(m, elec_array.freqsElectrodos, nChann, max_di)
combinaciones = buscarMejoresCombinacionesMelodia(elec_array.freqsElectrodos, m , nChann, nChann, max_di)

In [ ]:
import pickle
if(not combinaciones): Exception('No sigass')
with open('una nota medel temp.pkl', 'wb') as f:  # Python 3: open(..., 'rb')
    pickle.dump(combinaciones, f)

In [ ]:
elec_array = GENERIC_MEDEL3
import pickle
with open('una nota medel temp.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
    combinaciones = pickle.load(f)

In [ ]:
elec_array.generarTodasSeniales(combinaciones[3][0], sr = 44100)

Cochlear  (quedo viejo, para best_match v2 o v3 seguramente)

In [ ]:
patron =  encontrarMejorMatchMelodia(m, GENERIC_COCHLEAR3.freqsElectrodos, nChann, max_di)

In [ ]:
for p in patron:
    n = len(p.canalesActivos())
    print(n)

In [ ]:
elec_array = GENERIC_COCHLEAR3

nChann = 6
max_di = 300

#patron =  encontrarMejorMatchMelodia(m, elec_array.freqsElectrodos, nChann, max_di)
out_intermedio = elec_array.generarSenialMejorMatchMelodia(m, nChann,max_di, sr = IMPLANT_SR)

print('Melo para el implante')
Audio(out_intermedio, rate = IMPLANT_SR)

out_final = elec_array.CIS(out_intermedio, input_sr = IMPLANT_SR)

print('Melo que se escucha con el implante')
Audio(out_final, rate = OUT_SR)

## Con algunos armonicos

# Prueba 7 - Comparación auditiva del espectro de las notas originales vs el espectro que les elijo representar

Emm ya está en las demas pruebas esto

In [ ]:
amplitudes_armonicos = saw

m = Melodia.leerMidi(r"C:\Users\Graciela\Desktop\Cochlear Implants\CODE ultima version\midis\Pablo Pasea.mid", amplitudes_armonicos)
out = m.sintetizar(44100, vibratoDepth = 0.2, vibratoFreq = 2)

print('Melo original: Pablo pasea')
Audio(out, rate = 44100)

# Prueba 8 - Comparar con y sin Vibrato

no importante ahora considero

# Prueba 9 - Como se escuchan notas solo con los N armonicos superiores

Estas pruebas de concepto son importantes para ver si el principio que quiero aplicar con implantes funciona con oyentes

Resultado: Se escucha re bien, se entiende la melodía *para alguien con entrenamiento previo*

In [ ]:
import import_ipynb
#from CIS_6 import *
from CIS_6_Best_Match_v4 import *

In [ ]:
def prueba_armonicos_sup(amplitudes_armonicos):
    m = Melodia.leerMidi(r"midis\Pablo Pasea.mid", amplitudes_armonicos)
    out = m.sintetizar(44100, vibratoDepth = 0.2, vibratoFreq = 2)

    print('Pablo pasea')
    display(Audio(out, rate = 44100))

Armonicos 3-5

In [ ]:
amplitudes_armonicos = np.array([0,0,1,1,0.8])
prueba_armonicos_sup(amplitudes_armonicos)


Armonicos 4-5

In [ ]:
amplitudes_armonicos = np.array([0,0,0,1,0.8])
prueba_armonicos_sup(amplitudes_armonicos)


Armonico 5 solo

In [ ]:
amplitudes_armonicos = np.array([0,0,0,0,1,0.05])
prueba_armonicos_sup(amplitudes_armonicos)

Armonicos 5-6

In [ ]:
amplitudes_armonicos = np.array([0,0,0,0,1,0.8])
prueba_armonicos_sup(amplitudes_armonicos)


Armónicos 5-8

In [ ]:
amplitudes_armonicos = np.array([0,0,0,0,1,1,1,1])
prueba_armonicos_sup(amplitudes_armonicos)


Armónicos 5-10

In [ ]:
amplitudes_armonicos = np.array([0,0,0,0,1,1,1,1,1,1])
prueba_armonicos_sup(amplitudes_armonicos)


Armónicos 7-10

In [ ]:
amplitudes_armonicos = np.array([0,0,0,0,0,0,1,1,1,1,1])
prueba_armonicos_sup(amplitudes_armonicos)


Ok. Se percibe una melodía en esa octava. Es tremendo. Faltan los 6 primeros armonicos y aun asi se escucha clarita la altura...

 Ahora a trasponer 5J y armonicos 5-8

In [ ]:
amplitudes_armonicos = np.array([0,0,0,0,1,1,1,1])

m = Melodia.leerMidi(r"midis\Pablo Pasea.mid", amplitudes_armonicos)
m.transportar(1.5) #una 5ta arriba
out = m.sintetizar(44100, vibratoDepth = 0.2, vibratoFreq = 2)

print('Melo original - Pablo pasea + 5J')
Audio(out, rate = 44100)

Para nosotros pasa a entrar en el limite de lo comodo ya. Una 8va mas y es inescuchable

Ahora armonicos 3-5 + 5J

In [ ]:
amplitudes_armonicos = np.array([0,0,1,1,1])

m = Melodia.leerMidi(r"midis\Pablo Pasea.mid", amplitudes_armonicos)
m.transportar(1.5) #una 5ta arriba
out = m.sintetizar(44100, vibratoDepth = 0.2, vibratoFreq = 2)

print('Melo original - Pablo pasea + 5J')
Audio(out, rate = 44100)

Mejora. A partir de esto se podría definir un ámbito ideal para la melodía

In [ ]:
amplitudes_armonicos = np.array([1/k for k in range(1,10)])

m = Melodia.leerMidi(r"midis\Pablo Pasea.mid", amplitudes_armonicos)
m.transportar(0.75) 
out = m.sintetizar(44100, vibratoDepth = 0.2, vibratoFreq = 2)

print('Melo original - Pablo pasea -4J')
Audio(out, rate = 44100)

## Prueba 9b - timbre cambiante todo el tiempo (ponele nota a nota)

Usando sawwaves de base

In [ ]:
amplitudes_armonicos = np.array([0,0,1/3,1/4,1/5])  #3-5
m = Melodia.leerMidi(r"midis\Pablo Pasea.mid", amplitudes_armonicos)
m.transportar(0.66) #una 5ta abajo
out1 = m.sintetizar(44100, vibratoDepth = 0.2, vibratoFreq = 2)

amplitudes_armonicos = np.array([0,0,0,0+1/4,1/5,1/6,1/7])  #4-8
m = Melodia.leerMidi(r"midis\Pablo Pasea.mid", amplitudes_armonicos)
m.transportar(0.66) #una 5ta abajo
out2 = m.sintetizar(44100, vibratoDepth = 0.2, vibratoFreq = 2)

amplitudes_armonicos = np.array([0,0,0,0,1/5,1/6,1/7])  #5-7
m = Melodia.leerMidi(r"midis\Pablo Pasea.mid", amplitudes_armonicos)
m.transportar(0.66) #una 5ta abajo
out3 = m.sintetizar(44100, vibratoDepth = 0.2, vibratoFreq = 2)


amplitudes_armonicos = np.array([0,0,0,0,0,1/6,1/7,1/8,1/9,1/10])  #6-10
m = Melodia.leerMidi(r"midis\Pablo Pasea.mid", amplitudes_armonicos)
m.transportar(0.66) #una 5ta abajo
out4 = m.sintetizar(44100, vibratoDepth = 0.2, vibratoFreq = 2)


In [ ]:
out = np.zeros(len(out1))
outs = [out1, out2, out3, out4]
i = 0
k = 0 #out_k

import random
while i < len(out):
    j = random.choice([44100, 44100*2, 44100//2])
    out[i:i+j] = outs[k][i:i+j] 
    i += j
    k = (k+1) % 4

print('Pablo pasea - Timbres mezclados')
display(Audio(out, rate= 44100))

Usando dirac deltas de base

In [ ]:
amplitudes_armonicos = np.array([0,0,1,1,1])  #3-5
m = Melodia.leerMidi(r"midis\Pablo Pasea.mid", amplitudes_armonicos)
m.transportar(0.66) #una 5ta abajo
out1 = m.sintetizar(44100, vibratoDepth = 0.2, vibratoFreq = 2)

amplitudes_armonicos = np.array([0,0,0,0,1,1,1,1])  #4-8
m = Melodia.leerMidi(r"midis\Pablo Pasea.mid", amplitudes_armonicos)
m.transportar(0.66) #una 5ta abajo
out2 = m.sintetizar(44100, vibratoDepth = 0.2, vibratoFreq = 2)

amplitudes_armonicos = np.array([0,0,0,0,1,1,1])  #5-7
m = Melodia.leerMidi(r"midis\Pablo Pasea.mid", amplitudes_armonicos)
m.transportar(0.66) #una 5ta abajo
out3 = m.sintetizar(44100, vibratoDepth = 0.2, vibratoFreq = 2)


amplitudes_armonicos = np.array([0,0,0,0,0,1,1,1,1,1])  #6-10
m = Melodia.leerMidi(r"midis\Pablo Pasea.mid", amplitudes_armonicos)
m.transportar(0.66) #una 5ta abajo
out4 = m.sintetizar(44100, vibratoDepth = 0.2, vibratoFreq = 2)


In [ ]:
out = np.zeros(len(out1))
outs = [out1, out2, out3, out4]
i = 0
k = 0 #out_k

import random
while i < len(out):
    j = random.choice([44100, 44100*2, 44100//2])
    out[i:i+j] = outs[k][i:i+j] 
    i += j
    k = (k+1) % 4

print('Pablo pasea - Timbres mezclados')
display(Audio(out, rate= 44100))

A estos ultimos tres, Axel dijo: "Suenan HORRIBLES, muy molesto y muuy agudo". Pero si le preguntas dice que lo puede considerar notas y que podria ser musica. No nota ninguna estructura interna en el audio.

Igual ojo que dijo esto cuando la sintesis era una 8va arriba por error mio y si, era insufrible - sospecho que hasta mas de una 8va quizas

## 9C : Fundamental ausente (pero armonicos no contiguos)

In [ ]:
amplitudes_armonicos = np.array([0,0,1,0,1])
prueba_armonicos_sup(amplitudes_armonicos)


In [ ]:
amplitudes_armonicos = np.array([0,1,0,0,1,1])
prueba_armonicos_sup(amplitudes_armonicos)


In [ ]:
amplitudes_armonicos = np.array([0,0,1,0,0,1,0,1])
prueba_armonicos_sup(amplitudes_armonicos)


In [ ]:
amplitudes_armonicos = np.array([0,1,0,0,0,1,0,1])
prueba_armonicos_sup(amplitudes_armonicos)


In [ ]:
amplitudes_armonicos = np.array([0,1,0,0,0,1,1,1])
prueba_armonicos_sup(amplitudes_armonicos)


In [ ]:
amplitudes_armonicos = np.array([0,0,0,1,0,1,0,1]) #solo pares, suena 8va arriba
prueba_armonicos_sup(amplitudes_armonicos)


In [ ]:
amplitudes_armonicos = np.array([0,0,0,1,0,1,0,0,1]) #casi solo pares
prueba_armonicos_sup(amplitudes_armonicos)


In [ ]:
amplitudes_armonicos = np.array([0,0,0,1,0,0,1,1]) #casi solo pares
prueba_armonicos_sup(amplitudes_armonicos)


In [ ]:
amplitudes_armonicos = np.array([0,0,1,1,0,0,0,1,0,1]) #casi solo pares
prueba_armonicos_sup(amplitudes_armonicos)


# Prueba 10 - Desafinación de parciales

In [ ]:
def probar_desafinaciones_pablopasea(desafinaciones, amplitudes_armonicos):

    m = Melodia.leerMidi(r"midis\Pablo Pasea.mid", amplitudes_armonicos, desafinaciones_cents = desafinaciones)
    m.transportar(1.5) #una 5ta arriba
    out = m.sintetizar(44100, vibratoDepth = 0.2, vibratoFreq = 2)
    display(Audio(out, rate = 44100))

Acá pruebo desafinando algunos cents los parciales de una nota y viendo como se perciben

In [ ]:
amplitudes_armonicos = np.array([1,1])
desafinaciones = np.array([-20,0]) #cents
print('-20 cents')
probar_desafinaciones_pablopasea(desafinaciones, amplitudes_armonicos)

In [ ]:
amplitudes_armonicos = np.array([1,1/2, 1/3, 1/4, 1/5])
desafinaciones = np.array([-50,0,0,0,0]) #cents
print('-50 cents')
probar_desafinaciones_pablopasea(desafinaciones, amplitudes_armonicos)

In [ ]:
amplitudes_armonicos = np.array([1,1/2, 1/3, 1/4, 1/5])
desafinaciones = np.array([-20,-20,0,0,0]) #cents
probar_desafinaciones_pablopasea(desafinaciones, amplitudes_armonicos)

In [ ]:
amplitudes_armonicos = np.array([1,1/2, 1/3, 1/4, 1/5])
desafinaciones = np.array([-20,10,10,-10,-20]) #cents
probar_desafinaciones_pablopasea(desafinaciones, amplitudes_armonicos)

In [ ]:
amplitudes_armonicos = np.array([1,1/2, 1/3, 1/4, 1/5])
desafinaciones = np.array([20,10,-10,-10,0]) #cents

probar_desafinaciones_pablopasea(desafinaciones, amplitudes_armonicos)

In [ ]:
amplitudes_armonicos = np.array([1,1/2, 1/3, 1/4, 1/5])
desafinaciones = np.array([-20,20,-20,20,-20]) #cents

probar_desafinaciones_pablopasea(desafinaciones, amplitudes_armonicos)

In [ ]:
amplitudes_armonicos = np.array([1,1/2, 1/3, 1/4, 1/5])
desafinaciones = np.array([-30,30,-30,30,-30]) #cents

probar_desafinaciones_pablopasea(desafinaciones, amplitudes_armonicos)

In [ ]:
amplitudes_armonicos = np.array([1,1/2, 1/3, 1/4, 1/5])
desafinaciones = np.array([30,-30,30,-30,30]) #cents

probar_desafinaciones_pablopasea(desafinaciones, amplitudes_armonicos)

In [ ]:
amplitudes_armonicos = np.array([1,1/2, 1/3, 1/4, 1/5])
desafinaciones = np.array([-50,50,-50,50,-50]) #cents

probar_desafinaciones_pablopasea(desafinaciones, amplitudes_armonicos)

In [ ]:
amplitudes_armonicos = np.array([1,1/2, 1/3, 1/4, 1/5])
desafinaciones = np.array([50,-50,50,-50,50]) #cents

probar_desafinaciones_pablopasea(desafinaciones, amplitudes_armonicos)

Ok. Para desafinaciones "regulares" lo escucho "pasable". No lindo, pero se entiende todavia la melodia. Creo yo. A ver ahora con randoms

## Prueba 10b - Desafinaciones random

In [ ]:
import random
def probar_desafinaciones_random_pablopasea(amplitudes_armonicos, dmax =50):

    desafinaciones = []
    for i in range(len(amplitudes_armonicos)):
        desafinaciones.append(random.randint(-dmax,dmax))
    m = Melodia.leerMidi(r"midis\Pablo Pasea.mid", amplitudes_armonicos, desafinaciones_cents = desafinaciones)
    m.transportar(1.5) #una 5ta arriba
    print(desafinaciones)
    out = m.sintetizar(44100, vibratoDepth = 0.2, vibratoFreq = 2)
    display(Audio(out, rate = 44100))

In [ ]:
amplitudes_armonicos = np.array([1,1/2, 1/3, 1/4, 1/5])
for _ in range(5):
    probar_desafinaciones_random_pablopasea(amplitudes_armonicos, dmax = 50)

Y ahora lo mas importante. Con fundamental ausente


Pero OJO!!! Por como lo hice en su momento, las amplitudes no pueden ser exactamente 0 (porque si no desp se pierde que numero de armonico era). Setearlas a casi 0. Eso deberia arreglarlo...

In [ ]:
amplitudes_armonicos = np.array([0.001, 0.001,0.001, 1/4, 1/5, 1/6])
for _ in range(5):
    probar_desafinaciones_random_pablopasea(amplitudes_armonicos, dmax = 50)

Ok. El caso real es más complejo por lo siguiente: Las desafinaciones cambian en cada nota (ej las desafinaciones de C4 son siempre las mismas, pero distintas a D4). Y son bandas de ruido y no sinusides. Y las frecs están muucho más discretizadas

A partir de v4, en la tesis, trate de que  el timbre (amps) sea el mismo en toda la melo como acá. Antes, el timbre cambiaba nota a nota. 
Qué es mejor no sé. Pero se mantiene lo de que nota a nota las desafinaciones cambian a diferencia de este ejemplo

## Prueba 10c - Desafinacion random en cada nota

Ahora si aplico el comentario anterior. Para nosotros deberia estar todo bien por lo aprendido, por ej escuchamos alturas en notas de distintas campanas

In [ ]:
import random
def probar_desafinaciones_random_notaanota_pablopasea(amplitudes_armonicos, dmax =50):
    m = Melodia.leerMidi(r"midis\Pablo Pasea.mid", amplitudes_armonicos)

    newNotas = []
    for nota in m.notas:
        desafinaciones = []
        for i in range(len(amplitudes_armonicos)):
            desafinaciones.append(random.randint(-dmax,dmax))
        print(desafinaciones)
        
        newNotas.append(Nota(nota.freq, nota.amps,nota.dur, desafinaciones_cents= desafinaciones, ti = nota.ti))

    m.notas = newNotas
    #Ni idea por que no podia hacer nota = Nota(...) en el loop. No se guardaba sobre lo viejo final

    out = m.sintetizar(44100, vibratoDepth = 0.2, vibratoFreq = 2)
    display(Audio(out, rate = 44100))

Con fundamental ausente de una

In [ ]:
amplitudes_armonicos = np.array([0.001, 0.001,0.001, 1/4, 1/5, 1/6])
probar_desafinaciones_random_notaanota_pablopasea(amplitudes_armonicos, dmax = 80)

Esto si se acerca mas a lo que teoricamente yo predigo en la tesis que se escucharia con el implante

## 10d - Con Noise

In [ ]:
import random
def probar_desafinaciones_random_notaanota_pablopasea_noise(amplitudes_armonicos, dmax =50, bw = 1.5):
    m = Melodia.leerMidi(r"midis\Pablo Pasea.mid", amplitudes_armonicos)

    newNotas = []
    for nota in m.notas:
        desafinaciones = []
        for i in range(len(amplitudes_armonicos)):
            desafinaciones.append(random.randint(-dmax,dmax))
        print(desafinaciones)
        
        newNotas.append(Nota(nota.freq, nota.amps,nota.dur, desafinaciones_cents= desafinaciones, ti = nota.ti))

    m.notas = newNotas
    #Ni idea por que no podia hacer nota = Nota(...) en el loop. No se guardaba sobre lo viejo final

    out = m.sintetizarNoise(44100, bw = bw)
    display(Audio(out, rate = 44100))

In [ ]:
amplitudes_armonicos = np.array([0.001, 0.001,0.001, 1/4, 1/5, 1/6])
probar_desafinaciones_random_notaanota_pablopasea_noise(amplitudes_armonicos, dmax = 120, bw = 1.5)

# Prueba 11 - Prueba reasignacion con nota sola

## Cochlear

Para todas estas pruebas, la formula de puntaje utilizada fue

     (dist/50)**2 * (2**armonico_i) 

In [ ]:
pablo = Melodia.leerMidi(r"midis\Pablo Pasea.mid", [1]*8)
pablo.notas = [pablo.notas[0]]

In [ ]:
#Proceso. Tarda como 2hs para array cochlear
nota_puntuada_porclase = buscarMejoresCombinacionesMelodia(CIS_6.Cochlear_Axel_Der.freqsElectrodos, pablo , 3, 6, 100)

In [ ]:
#Guardo todo este resultado
import pickle
with open('output una nota Cochlear Derecho.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump(nota_puntuada_porclase, f)

In [ ]:
# Getting back the objects:
import pickle
with open('output una nota Cochlear Derecho.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
    nota_puntuada_porclase = pickle.load(f)

In [ ]:
nota = nota_puntuada_porclase[3][0]
nota.printInformeScores()

sr = 44100
print('Nota que quedó elegida como mejor')
display(Audio(nota.sintetizar(sr, vibratoDepth = 0.2, vibratoFreq = 1.5, fc = 1500), rate = sr, normalize= False))
print('Como se escucharía')
display(Audio(nota.sintetizarNoise(sr, bw = 1.5), rate = sr, normalize= False))


Escucho best matches por clase

In [ ]:
def reporteCompleto(clases_melo, clase, puesto):
    melo = clases_melo[clase][puesto]
    melo.printInformeScores()

    sr = 44100
    print('Nota que quedó elegida como mejor')
    display(CIS_6.Audio(melo.sintetizar(sr, vibratoDepth = 0.2, vibratoFreq = 1.5, fc = 1500), rate = sr, normalize= False))
    print('Como se escucharía')
    display(CIS_6.Audio(melo.sintetizarNoise(sr, bw = 1.5), rate = sr, normalize= False))


N = 3

Clase 4

In [ ]:
reporteCompleto(nota_puntuada_porclase, 4, 0)

N = 5

In [ ]:
reporteCompleto(nota_puntuada_porclase, 5, 0)

N = 6

In [ ]:
reporteCompleto(nota_puntuada_porclase, 6, 0)

Discutir en el informe los problemas de "sensacion de altura" con un implante. Que es super discutible y dificil. Pero que con melodias ahi quizas si

## Medel

Para estas pruebas, la formula de puntaje utilizada fue

     (dist)**2 

In [ ]:
pablo = Melodia.leerMidi(r"midis\Pablo Pasea.mid", [1]*8)
pablo.notas = [pablo.notas[0]]

#Proceso. Tarda como 2hs para array cochlear
nota_puntuada_porclase = buscarMejoresCombinacionesMelodia(CIS_6.GENERIC_MEDEL3.freqsElectrodos, pablo , 3, 6, 250)

In [ ]:
#Guardo todo este resultado
import pickle
with open('output una nota Medel.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump(nota_puntuada_porclase, f)

In [ ]:
# Getting back the objects:
import pickle
with open('output una nota Medel.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
    nota_puntuada_porclase = pickle.load(f)

reporteCompleto(nota_puntuada_porclase, 6, 0)

# Prueba 12 - Prueba reasignacion con melodia

funcion de puntaje utilizada... ni preguntes. Para coc debe ser la misma vieja que arriba, y medel la nueva

## Cochlaer pentatonica

In [ ]:
import import_ipynb
from CIS_6 import *
from CIS_6_Best_Match_v4 import *
import pickle

In [ ]:
def buscarMinMaxF0(melos):
    f0min = np.inf
    f0max = 0
    n_validos = 0
    for melo in melos:
        if(melo.score < np.inf):
            f0min = min(f0min, melo.notas[0].freq)
            f0max = max(f0max, melo.notas[0].freq)
            n_validos += 1
    return f0min, f0max, n_validos

In [ ]:
pablo = Melodia.leerMidi(r"midis\Pablo Pasea.mid", [1]*8)
pablo.notas = pablo.notas[1:3] + pablo.notas[4:6] + [pablo.notas[7]] #me quedo con las primeras notas nomas para acelerar esta prueba


#Proceso. Tarda mas de 17hs para array cochlear
melodias_puntuadas_porclase2 = buscarMejoresCombinacionesMelodia(CIS_6.Cochlear_Axel_Der.freqsElectrodos, pablo , 3, 6, 100)

#Guardo lo procesado. En realidad eran cinco notas. Pero "seis" en total con el primer Re que omití
with open('output seis notas Cochlear Derecho.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump(melodias_puntuadas_porclase2, f)

In [ ]:
# Getting back the objects:
with open('output seis notas Cochlear Derecho.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
    melodias_puntuadas_porclase2 = pickle.load(f)

In [ ]:
buscarMinMaxF0(melodias_puntuadas_porclase2[6])

In [ ]:
len(melodias_puntuadas_porclase2[6][0].notas) 

In [ ]:
np.array(buscarMinMaxF0(melodias_puntuadas_porclase2[3]))*8/9 #el tono hacia abajo es porque esta melo no lleva el "Re" grave... solo los grados 23568. Lo compenso entonces "trucho"

### arpegio

In [ ]:
pablo = Melodia.leerMidi(r"midis\Pablo Pasea.mid", [1]*N_Mayus)
pablo.notas = [pablo.notas[0], pablo.notas[2], pablo.notas[4], pablo.notas[9]]

In [ ]:
#Proceso. Tarda mas de 17hs para array cochlear
arpegios_puntuados_porclase = buscarMejoresCombinacionesMelodia(CIS_6.Cochlear_Axel_Der.freqsElectrodos, pablo , 3, 6, 100)

In [ ]:
#Guardo lo procesado
with open('output arpegio Cochlear Derecho.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump(arpegios_puntuados_porclase, f)

In [ ]:
# Getting back the objects:
with open('output arpegio Cochlear Derecho.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
    arpegios_puntuados_porclase = pickle.load(f)

In [ ]:
buscarMinMaxF0(arpegios_puntuados_porclase[5])

In [ ]:
buscarMinMaxF0(arpegios_puntuados_porclase[4])

Que haya mas combinaciones validas para la clase 5 que la 4 me hace dudas mucho de estos resultados... Voy a tener que chequearlo de alguna manera

In [ ]:
arpegios_puntuados_porclase[6][75].printInformeScores()

Lindos, hermosos resultados... Solo hubiera cambiado la funcion de puntaje, y mirado las distancias

## Medel pentatonica MAL

Ojo: Todos estos res que siguen estan remal. Los pongo bien abajo

In [ ]:
pablo = Melodia.leerMidi(r"midis\Pablo Pasea.mid", [1]*N_Mayus)
penta = Melodia(pablo.notas[0:3] + pablo.notas[4:6] + [pablo.notas[7]]) #me quedo con las primeras notas nomas para acelerar esta prueba

todo fix el matcheo entre elecs y patrones....

y hacer un comentario o reprensar lo de electrodo asignado a frecuencia...

In [ ]:
#Proceso. Tarda mas de 17hs para array cochlear y 30min para Medel
melodias_puntuadas_porclase3 = buscarMejoresCombinacionesMelodia(GENERIC_MEDEL3.freqsElectrodos, penta , 3, 6, 200)

todo desp con otro score de solo distancias... y listo...

In [ ]:
#Guardo lo procesado
import pickle

with open('output siete notas Medel Flex28.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump(melodias_puntuadas_porclase3, f)

In [ ]:
# Getting back the objects:
import pickle
with open('output siete notas Medel Flex28.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
    melodias_puntuadas_porclase3 = pickle.load(f)

In [ ]:
reporteCompleto(melodias_puntuadas_porclase3, 6, 0)

--> Todas las configuraciones dieron error > 100cents

### Medel pentatonica BIEN

In [ ]:
#Proceso. Tarda mas de 17hs para array cochlear y 30min para Medel
melodias_puntuadas_porclase3 = buscarMejoresCombinacionesMelodia(GENERIC_MEDEL3.freqsElectrodos, penta , 3, 6, 200)

In [ ]:
buscarMinMaxF0(melodias_puntuadas_porclase3[3]) #unica valida. Todas las demás: 0

In [ ]:
#Proceso. Tarda mas de 17hs para array cochlear y 30min para Medel
melodias_puntuadas_porclase3 = buscarMejoresCombinacionesMelodia(GENERIC_MEDEL3.freqsElectrodos, penta , 3, 6, 250)

### Medel Arpegio

OJO: Ahora lo cambie en este bloque, pero todos los resultados que ya están, USAN UN ARPEGIO DE 3 NOTAS DISTINTAS: La ultima es igual. Me equivoqué yo al poner el indice y puse un 9

In [ ]:
pablo = Melodia.leerMidi(r"midis\Pablo Pasea.mid", [1]*N_Mayus)
arp = Melodia([pablo.notas[0], pablo.notas[2], pablo.notas[4], pablo.notas[7]])


In [ ]:
arpegiosmedel34 = buscarMejoresCombinacionesMelodia(GENERIC_MEDEL3.freqsElectrodos, arp , 3, 4, 500) #500... es una banda

#Guardo lo procesado
import pickle

with open('output arp Medel Flex28 dmax500.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump(arpegiosmedel34, f)

Ese lo ignoro porque puse dmax = 500. Vamos con menos

In [ ]:

#Proceso. Tarda mas de 17hs para array cochlear
arpegios_puntuados_porclase2 = buscarMejoresCombinacionesMelodia(GENERIC_MEDEL3.freqsElectrodos, arp , 3, 6, 200)

#Guardo lo procesado
with open('output arpegio Medel dmax 200.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump(arpegios_puntuados_porclase2, f)

In [ ]:
arpegiosmedel34 = buscarMejoresCombinacionesMelodia(GENERIC_MEDEL3.freqsElectrodos, arp , 3, 4, 200)

In [ ]:
arpegiosmedel56 = buscarMejoresCombinacionesMelodia(GENERIC_MEDEL3.freqsElectrodos, arp , 5, 6, 200)

In [ ]:
buscarMinMaxF0(arpegiosmedel34[3])

In [ ]:
buscarMinMaxF0(arpegiosmedel34[4])

In [ ]:
buscarMinMaxF0(arpegiosmedel56[5])

In [ ]:
buscarMinMaxF0(arpegiosmedel56[6])